# Module 3: NLP Fundamentals - Text Preprocessing & Classical Methods

## Why NLP Matters

Natural Language Processing (NLP) is the branch of AI concerned with giving computers the ability to understand, generate, and manipulate human language. From search engines and spam filters to chatbots and machine translation, NLP powers countless applications we use every day.

## Classical vs. Modern Approaches

In this module we focus on **classical NLP** techniques that form the foundation of the field:

| Classical (this module) | Modern (later modules) |
|------------------------|------------------------|
| Rule-based tokenization | Subword tokenization (BPE, WordPiece) |
| Bag-of-Words, TF-IDF | Word embeddings (Word2Vec, GloVe) |
| N-gram language models | Transformer language models |
| Feature engineering + classifiers | End-to-end deep learning |

Understanding classical methods is essential because:
1. They establish the **vocabulary and intuition** used throughout NLP.
2. They are **fast, interpretable, and effective** baselines.
3. Modern methods (embeddings, transformers) are designed to overcome their specific **limitations**, so knowing those limitations helps you appreciate why modern methods work.

**What we will cover:**
- Text preprocessing (tokenization, stop words, stemming, lemmatization)
- Bag-of-Words and TF-IDF representations
- N-grams
- Text classification with classical ML
- Limitations and the bridge to word vectors

## 1. Setup

In [ ]:
!pip install -q scikit-learn matplotlib numpy nltk

In [ ]:
import re
import string
import math
from collections import Counter, defaultdict

import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.datasets import fetch_20newsgroups

plt.style.use('ggplot')
%matplotlib inline

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

## 2. Text Preprocessing

Before we can do anything useful with text, we need to clean and normalize it. Raw text is messy: it has inconsistent casing, punctuation, special characters, and other noise. **Preprocessing** converts raw text into a clean, standardized form that algorithms can work with.

### Sample Corpus

We will use a small corpus of movie reviews throughout this module.

In [ ]:
corpus = [
    "The movie was absolutely fantastic! Great acting and a wonderful storyline.",
    "I hated this film. The plot was terrible and the acting was wooden.",
    "A masterpiece of modern cinema. The director's vision is breathtaking.",
    "Worst movie I've ever seen. Complete waste of time and money.",
    "The special effects were amazing, but the story felt flat and predictable.",
    "An emotional rollercoaster! The performances were Oscar-worthy and the music was hauntingly beautiful."
]

labels = [1, 0, 1, 0, 0, 1]  # 1 = positive, 0 = negative

for i, review in enumerate(corpus):
    sentiment = "positive" if labels[i] == 1 else "negative"
    print(f"[{sentiment}] {review}")

### 2.1 Lowercasing and Punctuation Removal

The simplest preprocessing steps are **lowercasing** (so "The" and "the" are treated identically) and **punctuation removal** (so "fantastic!" becomes "fantastic").

In [ ]:
sample = corpus[0]
print("Original:  ", sample)

# Lowercasing
lowered = sample.lower()
print("Lowered:   ", lowered)

# Punctuation removal using str.translate
no_punct = lowered.translate(str.maketrans('', '', string.punctuation))
print("No punct:  ", no_punct)

### 2.2 Tokenization Approaches

**Tokenization** is the process of splitting text into individual units (tokens). There are several approaches, each with different tradeoffs.

#### Approach 1: Whitespace Splitting
The simplest method -- just split on spaces. Fast, but does not handle punctuation attached to words.

In [ ]:
text = "The movie was absolutely fantastic! Great acting."

# Whitespace splitting
tokens_whitespace = text.split()
print("Whitespace split:", tokens_whitespace)
print("Notice 'fantastic!' still has punctuation attached.")

#### Approach 2: Regex Tokenization

We can use a regular expression to extract only sequences of word characters.

In [ ]:
# Regex tokenization: extract sequences of word characters
tokens_regex = re.findall(r'\b\w+\b', text.lower())
print("Regex tokens:", tokens_regex)

#### Approach 3: NLTK word_tokenize

NLTK's `word_tokenize` uses a sophisticated rule-based tokenizer (Punkt) that handles contractions, abbreviations, and other edge cases.

In [ ]:
text_with_contraction = "I've never seen a movie that wasn't at least somewhat entertaining."

# NLTK tokenization
tokens_nltk = word_tokenize(text_with_contraction)
print("NLTK tokens:", tokens_nltk)
print("\nNotice how NLTK splits contractions: I've -> ['I', \"'ve\"]")

# Compare all three on the same input
print("\n--- Comparison ---")
print("Whitespace: ", text_with_contraction.split())
print("Regex:      ", re.findall(r'\b\w+\b', text_with_contraction.lower()))
print("NLTK:       ", word_tokenize(text_with_contraction))

### 2.3 Preview: Subword Tokenization

Classical tokenizers split text into whole words. Modern NLP models (BERT, GPT) use **subword tokenization** which splits words into smaller pieces:

- **Byte-Pair Encoding (BPE)**: Used by GPT models. Starts with individual characters and iteratively merges the most frequent pairs.
- **WordPiece**: Used by BERT. Similar to BPE but uses a likelihood-based merging criterion.
- **SentencePiece**: Language-agnostic, works directly on raw text without pre-tokenization.

Example: the word "unhappiness" might be tokenized as `["un", "##happi", "##ness"]` by WordPiece.

Subword tokenization solves the **out-of-vocabulary (OOV) problem** -- any word can be represented as a combination of known subword units. We will explore this in depth in later modules on transformers.

## 3. Stop Words, Stemming & Lemmatization

### 3.1 Stop Words

**Stop words** are very common words ("the", "is", "at", "and") that typically carry little meaning on their own. Removing them reduces noise and dimensionality.

In [ ]:
stop_words = set(stopwords.words('english'))

print(f"NLTK has {len(stop_words)} English stop words.")
print("Examples:", sorted(list(stop_words))[:20])

# Filtering stop words
sample_tokens = word_tokenize(corpus[0].lower())
print(f"\nBefore filtering ({len(sample_tokens)} tokens): {sample_tokens}")

filtered = [w for w in sample_tokens if w not in stop_words and w.isalpha()]
print(f"After filtering  ({len(filtered)} tokens): {filtered}")

### 3.2 Stemming

**Stemming** reduces words to their root form by chopping off suffixes using heuristic rules. It is fast but sometimes produces non-words.

The most common stemmer is the **Porter Stemmer**.

In [ ]:
stemmer = PorterStemmer()

stem_examples = ['running', 'runs', 'ran', 'runner', 'better', 'playing', 
                 'played', 'happiness', 'beautiful', 'beautifully']

print("Porter Stemmer Examples:")
print("-" * 35)
for word in stem_examples:
    print(f"  {word:15s} -> {stemmer.stem(word)}")

### 3.3 Lemmatization

**Lemmatization** reduces words to their dictionary form (lemma) using vocabulary and morphological analysis. It is slower but produces real words.

The key advantage: with the correct **part-of-speech (POS) tag**, a lemmatizer can map "better" to "good" (adjective), which a stemmer cannot do.

In [ ]:
lemmatizer = WordNetLemmatizer()

lemma_examples = [
    ('running', 'v'),   # verb
    ('runs', 'v'),
    ('ran', 'v'),
    ('better', 'a'),    # adjective
    ('playing', 'v'),
    ('played', 'v'),
    ('happiness', 'n'), # noun
    ('beautiful', 'a'),
    ('mice', 'n'),
    ('geese', 'n'),
]

print("WordNet Lemmatizer Examples:")
print("-" * 45)
for word, pos in lemma_examples:
    lemma = lemmatizer.lemmatize(word, pos=pos)
    print(f"  {word:15s} (pos={pos}) -> {lemma}")

### 3.4 Stemming vs. Lemmatization: Side-by-Side Comparison

In [ ]:
comparison_words = ['running', 'better', 'mice', 'happiness', 'studies', 
                    'beautifully', 'geese', 'played', 'worse', 'feet']
pos_tags =         ['v',       'a',      'n',    'n',         'v',
                    'r',       'n',       'v',    'a',         'n']

print(f"{'Word':15s} {'Stemmed':15s} {'Lemmatized':15s}")
print("-" * 45)
for word, pos in zip(comparison_words, pos_tags):
    stemmed = stemmer.stem(word)
    lemmatized = lemmatizer.lemmatize(word, pos=pos)
    print(f"{word:15s} {stemmed:15s} {lemmatized:15s}")

print("\nKey takeaway: Lemmatization produces real dictionary words.")
print("Stemming is faster but can produce non-words (e.g., 'happi', 'beauti').")

## 4. Bag-of-Words

The **Bag-of-Words (BoW)** model represents text as a vector of word counts. It ignores word order entirely -- only the *frequency* of each word matters.

### 4.1 Implement from Scratch

Let us build a Bag-of-Words representation step by step.

In [ ]:
def build_vocabulary(documents):
    """Build a sorted vocabulary from a list of documents."""
    vocab = set()
    for doc in documents:
        tokens = re.findall(r'\b\w+\b', doc.lower())
        vocab.update(tokens)
    return sorted(vocab)


def bow_vectorize(documents, vocab):
    """Create count vectors for each document."""
    word_to_idx = {word: i for i, word in enumerate(vocab)}
    vectors = np.zeros((len(documents), len(vocab)), dtype=int)
    
    for doc_idx, doc in enumerate(documents):
        tokens = re.findall(r'\b\w+\b', doc.lower())
        for token in tokens:
            if token in word_to_idx:
                vectors[doc_idx, word_to_idx[token]] += 1
    return vectors


# Build vocabulary and vectors
vocab = build_vocabulary(corpus)
bow_matrix = bow_vectorize(corpus, vocab)

print(f"Vocabulary size: {len(vocab)}")
print(f"BoW matrix shape: {bow_matrix.shape}  (documents x vocabulary)")
print(f"\nFirst 15 vocabulary words: {vocab[:15]}")
print(f"\nDocument 0 vector (first 15 dims): {bow_matrix[0, :15]}")

### 4.2 Compare with sklearn CountVectorizer

In [ ]:
sklearn_cv = CountVectorizer()
sklearn_bow = sklearn_cv.fit_transform(corpus)

print(f"sklearn BoW matrix shape: {sklearn_bow.shape}")
print(f"sklearn vocabulary size:  {len(sklearn_cv.vocabulary_)}")
print(f"Our vocabulary size:      {len(vocab)}")
print(f"\nsklearn vocabulary (first 15): {list(sklearn_cv.get_feature_names_out())[:15]}")

# They should be very similar (sklearn may filter single-char tokens)
print(f"\nsklearn doc 0 vector (first 15): {sklearn_bow.toarray()[0, :15]}")

### 4.3 Visualize: Word Frequency Bar Charts

In [ ]:
# Total word frequencies across all documents
total_counts = bow_matrix.sum(axis=0)
word_freq = sorted(zip(vocab, total_counts), key=lambda x: x[1], reverse=True)

# Plot top 20 words
top_n = 20
words = [w for w, c in word_freq[:top_n]]
counts = [c for w, c in word_freq[:top_n]]

fig, ax = plt.subplots(figsize=(12, 5))
ax.bar(range(top_n), counts, color='steelblue')
ax.set_xticks(range(top_n))
ax.set_xticklabels(words, rotation=45, ha='right')
ax.set_xlabel('Word')
ax.set_ylabel('Frequency')
ax.set_title('Top 20 Words by Frequency (Bag-of-Words)')
plt.tight_layout()
plt.show()

print("Notice how common stop words like 'the', 'was', 'and' dominate the counts.")
print("This is why stop word removal and TF-IDF weighting are important!")

---

## Exercise 1: Build a Complete Text Preprocessing Pipeline

Implement a function `preprocess(text)` that applies the following steps in order:
1. Lowercase the text
2. Remove punctuation
3. Tokenize using NLTK `word_tokenize`
4. Remove stop words
5. Lemmatize each token (use POS='v' for verbs as a default -- this handles most inflections well)

In [ ]:
def preprocess(text):
    """
    Complete text preprocessing pipeline.
    
    Steps:
      1. Lowercase
      2. Remove punctuation
      3. Tokenize with word_tokenize
      4. Remove stop words
      5. Lemmatize (default POS='v')
    
    Returns: list of processed tokens
    """
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    
    # Step 1: Lowercase
    text = None  # TODO: lowercase the text
    
    # Step 2: Remove punctuation
    text = None  # TODO: remove punctuation using str.translate
    
    # Step 3: Tokenize
    tokens = None  # TODO: tokenize using word_tokenize
    
    # Step 4: Remove stop words (also keep only alphabetic tokens)
    tokens = None  # TODO: filter out stop words and non-alpha tokens
    
    # Step 5: Lemmatize
    tokens = None  # TODO: lemmatize each token with pos='v'
    
    return tokens


# Test it
for doc in corpus:
    print(preprocess(doc))

### Solution

In [ ]:
def preprocess(text):
    """
    Complete text preprocessing pipeline.
    
    Steps:
      1. Lowercase
      2. Remove punctuation
      3. Tokenize with word_tokenize
      4. Remove stop words
      5. Lemmatize (default POS='v')
    
    Returns: list of processed tokens
    """
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    
    # Step 1: Lowercase
    text = text.lower()
    
    # Step 2: Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Step 3: Tokenize
    tokens = word_tokenize(text)
    
    # Step 4: Remove stop words (also keep only alphabetic tokens)
    tokens = [t for t in tokens if t not in stop_words and t.isalpha()]
    
    # Step 5: Lemmatize
    tokens = [lemmatizer.lemmatize(t, pos='v') for t in tokens]
    
    return tokens


# Test it
print("Preprocessed corpus:")
print("-" * 60)
for i, doc in enumerate(corpus):
    processed = preprocess(doc)
    print(f"Doc {i}: {processed}")

## 5. TF-IDF

### 5.1 The Problem with Raw Counts

Bag-of-Words treats all words equally. But common words like "the" and "was" appear everywhere and carry little discriminative information. **TF-IDF** (Term Frequency - Inverse Document Frequency) solves this by weighting words based on how *informative* they are.

### 5.2 The Formula

**TF (Term Frequency):** How often does the term appear in this document?

$$\text{TF}(t, d) = \frac{\text{count of } t \text{ in } d}{\text{total terms in } d}$$

**IDF (Inverse Document Frequency):** How rare is this term across all documents?

$$\text{IDF}(t) = \log\left(\frac{N}{1 + \text{df}(t)}\right) + 1$$

where $N$ is the total number of documents and $\text{df}(t)$ is the number of documents containing term $t$. The "+1" prevents division by zero and ensures non-negative weights.

**TF-IDF:**

$$\text{TF-IDF}(t, d) = \text{TF}(t, d) \times \text{IDF}(t)$$

Words that appear frequently in a specific document but rarely in other documents get high TF-IDF scores.

### 5.3 Implement TF-IDF from Scratch

In [ ]:
def compute_tf(document_tokens):
    """Compute term frequency for a single tokenized document."""
    tf = Counter(document_tokens)
    total_terms = len(document_tokens)
    # Normalize by total number of terms
    for term in tf:
        tf[term] = tf[term] / total_terms
    return tf


def compute_idf(corpus_tokens):
    """Compute inverse document frequency for all terms in the corpus."""
    N = len(corpus_tokens)
    idf = {}
    
    # Count how many documents contain each term
    all_terms = set()
    for doc_tokens in corpus_tokens:
        all_terms.update(doc_tokens)
    
    for term in all_terms:
        df = sum(1 for doc_tokens in corpus_tokens if term in doc_tokens)
        idf[term] = math.log(N / (1 + df)) + 1
    
    return idf


def compute_tfidf_from_scratch(corpus):
    """
    Compute TF-IDF matrix from a list of raw text documents.
    Returns: (tfidf_matrix, vocabulary)
    """
    # Tokenize all documents
    corpus_tokens = [re.findall(r'\b\w+\b', doc.lower()) for doc in corpus]
    
    # Build vocabulary
    vocab = sorted(set(token for doc in corpus_tokens for token in doc))
    word_to_idx = {w: i for i, w in enumerate(vocab)}
    
    # Compute IDF
    idf = compute_idf(corpus_tokens)
    
    # Compute TF-IDF for each document
    tfidf_matrix = np.zeros((len(corpus), len(vocab)))
    for doc_idx, doc_tokens in enumerate(corpus_tokens):
        tf = compute_tf(doc_tokens)
        for term, tf_val in tf.items():
            col_idx = word_to_idx[term]
            tfidf_matrix[doc_idx, col_idx] = tf_val * idf[term]
    
    return tfidf_matrix, vocab


# Compute TF-IDF from scratch
tfidf_scratch, vocab_scratch = compute_tfidf_from_scratch(corpus)

print(f"TF-IDF matrix shape: {tfidf_scratch.shape}")
print(f"Vocabulary size: {len(vocab_scratch)}")
print(f"\nDoc 0 - top 5 TF-IDF terms:")
doc0_scores = list(zip(vocab_scratch, tfidf_scratch[0]))
doc0_scores.sort(key=lambda x: x[1], reverse=True)
for word, score in doc0_scores[:5]:
    print(f"  {word:15s} {score:.4f}")

### 5.4 Compare with sklearn TfidfVectorizer

In [ ]:
sklearn_tfidf = TfidfVectorizer(norm=None)  # norm=None to compare raw values
sklearn_tfidf_matrix = sklearn_tfidf.fit_transform(corpus)

print(f"sklearn TF-IDF matrix shape: {sklearn_tfidf_matrix.shape}")

# Show top terms for doc 0 from sklearn
feature_names = sklearn_tfidf.get_feature_names_out()
doc0_sklearn = list(zip(feature_names, sklearn_tfidf_matrix.toarray()[0]))
doc0_sklearn.sort(key=lambda x: x[1], reverse=True)

print(f"\nsklearn Doc 0 - top 5 TF-IDF terms:")
for word, score in doc0_sklearn[:5]:
    print(f"  {word:15s} {score:.4f}")

print("\nNote: Values differ slightly because sklearn uses raw counts")
print("for TF (not normalized by doc length) by default.")
print("The relative ranking of terms should be similar.")

### 5.5 TF-IDF vs. Raw Counts: Visual Comparison

In [ ]:
# Compare raw counts vs TF-IDF for document 0
doc_idx = 0

# Get raw counts
raw_counts = bow_matrix[doc_idx]
raw_top = sorted(zip(vocab, raw_counts), key=lambda x: x[1], reverse=True)[:10]

# Get TF-IDF
tfidf_scores = tfidf_scratch[doc_idx]
tfidf_top = sorted(zip(vocab_scratch, tfidf_scores), key=lambda x: x[1], reverse=True)[:10]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Raw counts
axes[0].barh([w for w, c in raw_top], [c for w, c in raw_top], color='steelblue')
axes[0].set_title(f'Document {doc_idx}: Raw Word Counts')
axes[0].set_xlabel('Count')
axes[0].invert_yaxis()

# TF-IDF
axes[1].barh([w for w, c in tfidf_top], [c for w, c in tfidf_top], color='coral')
axes[1].set_title(f'Document {doc_idx}: TF-IDF Scores')
axes[1].set_xlabel('TF-IDF Score')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

print("Key observation: TF-IDF boosts content-specific words and")
print("down-weights words that appear across many documents.")

---

## Exercise 2: Implement TF-IDF from Scratch

Implement the function `compute_tfidf(corpus)` that:
1. Tokenizes each document (lowercase, extract word characters)
2. Computes TF for each document
3. Computes IDF across the corpus
4. Returns a TF-IDF matrix (numpy array) and the vocabulary

Then compare your results with sklearn's `TfidfVectorizer`.

In [ ]:
def compute_tfidf(corpus):
    """
    Compute TF-IDF matrix from scratch.
    
    Args:
        corpus: list of raw text strings
    
    Returns:
        tfidf_matrix: numpy array of shape (n_docs, vocab_size)
        vocabulary: sorted list of terms
    """
    # Step 1: Tokenize
    tokenized = None  # TODO: tokenize each document (lowercase, word chars only)
    
    # Step 2: Build vocabulary
    vocabulary = None  # TODO: sorted list of all unique terms
    word_to_idx = None  # TODO: dict mapping word -> column index
    
    # Step 3: Compute IDF
    N = len(corpus)
    idf = {}  # TODO: compute IDF for each term using log(N / (1 + df)) + 1
    
    # Step 4: Compute TF-IDF matrix
    tfidf_matrix = None  # TODO: fill in TF * IDF values
    
    return tfidf_matrix, vocabulary


# Test and compare with sklearn
my_tfidf, my_vocab = compute_tfidf(corpus)

# Print results
if my_tfidf is not None:
    print(f"Your TF-IDF matrix shape: {my_tfidf.shape}")
    print(f"Vocabulary size: {len(my_vocab)}")
else:
    print("TODO: Complete the implementation above!")

### Solution

In [ ]:
def compute_tfidf(corpus):
    """
    Compute TF-IDF matrix from scratch.
    
    Args:
        corpus: list of raw text strings
    
    Returns:
        tfidf_matrix: numpy array of shape (n_docs, vocab_size)
        vocabulary: sorted list of terms
    """
    # Step 1: Tokenize
    tokenized = [re.findall(r'\b\w+\b', doc.lower()) for doc in corpus]
    
    # Step 2: Build vocabulary
    vocabulary = sorted(set(tok for doc in tokenized for tok in doc))
    word_to_idx = {w: i for i, w in enumerate(vocabulary)}
    
    # Step 3: Compute IDF
    N = len(corpus)
    idf = {}
    for term in vocabulary:
        df = sum(1 for doc in tokenized if term in doc)
        idf[term] = math.log(N / (1 + df)) + 1
    
    # Step 4: Compute TF-IDF matrix
    tfidf_matrix = np.zeros((N, len(vocabulary)))
    for doc_idx, doc_tokens in enumerate(tokenized):
        tf = Counter(doc_tokens)
        n_terms = len(doc_tokens)
        for term, count in tf.items():
            col = word_to_idx[term]
            tfidf_matrix[doc_idx, col] = (count / n_terms) * idf[term]
    
    return tfidf_matrix, vocabulary


# Compute and display
my_tfidf, my_vocab = compute_tfidf(corpus)
print(f"TF-IDF matrix shape: {my_tfidf.shape}")
print(f"Vocabulary size: {len(my_vocab)}")

# Compare with sklearn
print("\n--- Comparison: Top terms for Document 0 ---")
print(f"{'Term':15s} {'Ours':>10s}  {'sklearn':>10s}")
print("-" * 40)

sklearn_tv = TfidfVectorizer(norm=None)
sklearn_mat = sklearn_tv.fit_transform(corpus).toarray()
sklearn_vocab = sklearn_tv.get_feature_names_out()

# Top 8 from our implementation
our_top = sorted(zip(my_vocab, my_tfidf[0]), key=lambda x: x[1], reverse=True)[:8]
for word, score in our_top:
    # Find sklearn score for same word
    sk_idx = list(sklearn_vocab).index(word) if word in sklearn_vocab else -1
    sk_score = sklearn_mat[0, sk_idx] if sk_idx >= 0 else 0.0
    print(f"{word:15s} {score:10.4f}  {sk_score:10.4f}")

print("\nNote: Rankings should be similar even if absolute values differ")
print("(sklearn uses raw counts for TF, we normalize by document length).")

## 6. N-grams

So far, our representations treat each word independently. **N-grams** capture short sequences of consecutive words, preserving some local word order.

- **Unigram** (n=1): individual words -- "the", "movie", "was"
- **Bigram** (n=2): pairs of words -- "the movie", "movie was"
- **Trigram** (n=3): triples -- "the movie was", "movie was great"

### 6.1 Implement N-gram Extraction

In [ ]:
def extract_ngrams(tokens, n):
    """Extract n-grams from a list of tokens."""
    return [tuple(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]


# Example
sample_text = "the movie was absolutely fantastic and the acting was wonderful"
tokens = sample_text.split()

print("Tokens:", tokens)
print()

for n in [1, 2, 3]:
    ngrams = extract_ngrams(tokens, n)
    label = {1: 'Unigrams', 2: 'Bigrams', 3: 'Trigrams'}[n]
    print(f"{label} (n={n}):")
    for ng in ngrams:
        print(f"  {' '.join(ng)}")
    print()

### 6.2 Most Common Bigrams and Trigrams in Our Corpus

In [ ]:
# Collect n-grams across the entire corpus
all_bigrams = []
all_trigrams = []

for doc in corpus:
    tokens = re.findall(r'\b\w+\b', doc.lower())
    all_bigrams.extend(extract_ngrams(tokens, 2))
    all_trigrams.extend(extract_ngrams(tokens, 3))

# Most common
print("Top 10 Bigrams:")
for bigram, count in Counter(all_bigrams).most_common(10):
    print(f"  {' '.join(bigram):25s} (count: {count})")

print("\nTop 10 Trigrams:")
for trigram, count in Counter(all_trigrams).most_common(10):
    print(f"  {' '.join(trigram):30s} (count: {count})")

## 7. Sentiment Classification: TF-IDF + Logistic Regression

Now let us put everything together and build a real text classifier. We will use sklearn's built-in **20 Newsgroups** dataset (no external downloads needed) and classify posts into categories.

### 7.1 Load the Dataset

In [ ]:
# Pick 3 distinct categories for a manageable classification task
categories = ['sci.space', 'rec.sport.hockey', 'comp.graphics']

newsgroups_train = fetch_20newsgroups(
    subset='train', categories=categories, remove=('headers', 'footers', 'quotes')
)
newsgroups_test = fetch_20newsgroups(
    subset='test', categories=categories, remove=('headers', 'footers', 'quotes')
)

print(f"Training samples: {len(newsgroups_train.data)}")
print(f"Test samples:     {len(newsgroups_test.data)}")
print(f"Categories:       {newsgroups_train.target_names}")

# Show a sample
print(f"\n--- Sample document (category: {newsgroups_train.target_names[newsgroups_train.target[0]]}) ---")
print(newsgroups_train.data[0][:500])

### 7.2 TF-IDF Vectorization + Logistic Regression

In [ ]:
# Vectorize with TF-IDF
tfidf_vec = TfidfVectorizer(max_features=10000, stop_words='english')
X_train = tfidf_vec.fit_transform(newsgroups_train.data)
X_test = tfidf_vec.transform(newsgroups_test.data)
y_train = newsgroups_train.target
y_test = newsgroups_test.target

print(f"TF-IDF feature matrix: {X_train.shape}")

# Train Logistic Regression
clf = LogisticRegression(max_iter=1000, random_state=42)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print(f"\nAccuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=categories))

### 7.3 Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred)

fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
ax.set_title('Confusion Matrix: TF-IDF + Logistic Regression')
plt.colorbar(im, ax=ax)

# Add labels
tick_marks = range(len(categories))
ax.set_xticks(tick_marks)
ax.set_xticklabels(categories, rotation=45, ha='right')
ax.set_yticks(tick_marks)
ax.set_yticklabels(categories)

# Add numbers in cells
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, format(cm[i, j], 'd'),
                ha='center', va='center',
                color='white' if cm[i, j] > thresh else 'black')

ax.set_ylabel('True Label')
ax.set_xlabel('Predicted Label')
plt.tight_layout()
plt.show()

---

## Exercise 3: Bag-of-Words vs. TF-IDF Classifier Comparison

Train two classifiers on the 20 Newsgroups data:
1. **Model A**: `CountVectorizer` (Bag-of-Words) + `LogisticRegression`
2. **Model B**: `TfidfVectorizer` (TF-IDF) + `LogisticRegression`

Compare their accuracy. Which one performs better and why?

In [ ]:
# Exercise 3: Compare CountVectorizer vs TfidfVectorizer

# Model A: Bag-of-Words + Logistic Regression
bow_vectorizer = None  # TODO: create CountVectorizer(max_features=10000, stop_words='english')
X_train_bow = None     # TODO: fit_transform on training data
X_test_bow = None      # TODO: transform test data

clf_bow = None          # TODO: create and train LogisticRegression
accuracy_bow = None     # TODO: compute accuracy on test set

# Model B: TF-IDF + Logistic Regression
tfidf_vectorizer = None  # TODO: create TfidfVectorizer(max_features=10000, stop_words='english')
X_train_tfidf = None     # TODO: fit_transform on training data
X_test_tfidf = None      # TODO: transform test data

clf_tfidf = None          # TODO: create and train LogisticRegression
accuracy_tfidf = None     # TODO: compute accuracy on test set

# Compare
print(f"Bag-of-Words accuracy: {accuracy_bow}")
print(f"TF-IDF accuracy:       {accuracy_tfidf}")

### Solution

In [ ]:
# Model A: Bag-of-Words + Logistic Regression
bow_vectorizer = CountVectorizer(max_features=10000, stop_words='english')
X_train_bow = bow_vectorizer.fit_transform(newsgroups_train.data)
X_test_bow = bow_vectorizer.transform(newsgroups_test.data)

clf_bow = LogisticRegression(max_iter=1000, random_state=42)
clf_bow.fit(X_train_bow, y_train)
y_pred_bow = clf_bow.predict(X_test_bow)
accuracy_bow = accuracy_score(y_test, y_pred_bow)

# Model B: TF-IDF + Logistic Regression
tfidf_vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(newsgroups_train.data)
X_test_tfidf = tfidf_vectorizer.transform(newsgroups_test.data)

clf_tfidf = LogisticRegression(max_iter=1000, random_state=42)
clf_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = clf_tfidf.predict(X_test_tfidf)
accuracy_tfidf = accuracy_score(y_test, y_pred_tfidf)

# Compare
print(f"Bag-of-Words accuracy: {accuracy_bow:.4f}")
print(f"TF-IDF accuracy:       {accuracy_tfidf:.4f}")
print(f"Difference:            {accuracy_tfidf - accuracy_bow:+.4f}")

# Visual comparison
fig, ax = plt.subplots(figsize=(6, 4))
models = ['Bag-of-Words', 'TF-IDF']
accuracies = [accuracy_bow, accuracy_tfidf]
colors = ['steelblue', 'coral']
bars = ax.bar(models, accuracies, color=colors)
ax.set_ylabel('Accuracy')
ax.set_title('BoW vs TF-IDF: Classification Accuracy')
ax.set_ylim(0.8, 1.0)
for bar, acc in zip(bars, accuracies):
    ax.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.005,
            f'{acc:.4f}', ha='center', va='bottom', fontweight='bold')
plt.tight_layout()
plt.show()

print("\nTF-IDF typically outperforms raw BoW because it down-weights")
print("common terms that appear across many documents, giving more")
print("importance to discriminative, category-specific terms.")

---

## Exercise 4: Analyze N-gram Range Effects on Classifier Performance

Train TF-IDF + Logistic Regression classifiers with different `ngram_range` settings:
- `(1, 1)` -- unigrams only
- `(1, 2)` -- unigrams + bigrams
- `(1, 3)` -- unigrams + bigrams + trigrams

Plot the accuracies. Does including bigrams and trigrams help?

In [ ]:
# Exercise 4: N-gram range effects

ngram_ranges = [(1, 1), (1, 2), (1, 3)]
ngram_accuracies = []

for ngram_range in ngram_ranges:
    # TODO: Create TfidfVectorizer with the current ngram_range
    vectorizer = None  # TODO
    
    # TODO: Fit and transform
    X_tr = None  # TODO
    X_te = None  # TODO
    
    # TODO: Train classifier and compute accuracy
    model = None  # TODO
    acc = None    # TODO
    
    ngram_accuracies.append(acc)
    print(f"ngram_range={ngram_range}: accuracy={acc}")

# TODO: Plot the results

### Solution

In [ ]:
ngram_ranges = [(1, 1), (1, 2), (1, 3)]
ngram_labels = ['(1,1)\nUnigrams', '(1,2)\n+ Bigrams', '(1,3)\n+ Trigrams']
ngram_accuracies = []
ngram_feature_counts = []

for ngram_range in ngram_ranges:
    vectorizer = TfidfVectorizer(
        max_features=20000, stop_words='english', ngram_range=ngram_range
    )
    X_tr = vectorizer.fit_transform(newsgroups_train.data)
    X_te = vectorizer.transform(newsgroups_test.data)
    
    model = LogisticRegression(max_iter=1000, random_state=42)
    model.fit(X_tr, y_train)
    y_pred_ng = model.predict(X_te)
    acc = accuracy_score(y_test, y_pred_ng)
    
    ngram_accuracies.append(acc)
    ngram_feature_counts.append(X_tr.shape[1])
    print(f"ngram_range={str(ngram_range):8s}  features={X_tr.shape[1]:6d}  accuracy={acc:.4f}")

# Plot results
fig, ax1 = plt.subplots(figsize=(8, 5))

color1 = 'steelblue'
bars = ax1.bar(ngram_labels, ngram_accuracies, color=color1, alpha=0.8)
ax1.set_ylabel('Accuracy', color=color1)
ax1.set_title('Effect of N-gram Range on Classification Accuracy')
ax1.set_ylim(0.85, 1.0)
ax1.tick_params(axis='y', labelcolor=color1)

# Add accuracy labels on bars
for bar, acc in zip(bars, ngram_accuracies):
    ax1.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.003,
            f'{acc:.4f}', ha='center', va='bottom', fontweight='bold')

# Add feature count on secondary axis
ax2 = ax1.twinx()
color2 = 'coral'
ax2.plot(ngram_labels, ngram_feature_counts, 'o-', color=color2, linewidth=2, markersize=8)
ax2.set_ylabel('Number of Features', color=color2)
ax2.tick_params(axis='y', labelcolor=color2)

plt.tight_layout()
plt.show()

print("\nObservations:")
print("- Adding bigrams often provides a small accuracy boost by capturing")
print("  useful two-word phrases (e.g., 'space shuttle', 'graphics card').")
print("- Adding trigrams adds many more features but may not help further,")
print("  and can lead to overfitting on small datasets.")
print("- The tradeoff is always: more n-grams = more features = more memory + compute.")

## 8. Limitations of Classical NLP

The techniques we have covered in this module are powerful baselines, but they have fundamental limitations that motivate the move to modern deep learning approaches.

### No Semantic Understanding

Bag-of-Words and TF-IDF treat words as **atomic symbols**. They have no notion that "happy" and "joyful" are related, or that "bank" can mean a financial institution or a river bank. Two sentences can have completely different BoW representations but mean the same thing:

- "The film was excellent" vs. "The movie was superb"
- These share only "the" and "was" -- their BoW similarity is very low!

### Sparsity of Representations

With a vocabulary of 50,000 words, each document is represented as a 50,000-dimensional vector that is mostly zeros. This **sparsity** wastes memory, makes distances meaningless in high dimensions (the "curse of dimensionality"), and makes it hard for models to generalize.

### Vocabulary Mismatch Problem

If a word never appeared in the training data, it gets no representation at all. New words, misspellings, and domain-specific jargon are invisible to the model. This is the **out-of-vocabulary (OOV)** problem.

### No Word Order

"The dog bit the man" and "The man bit the dog" have identical BoW representations. N-grams partially address this but only capture very local context.

### The Bridge to Word Vectors (Module 4)

Word embeddings (Word2Vec, GloVe, and later Transformer-based embeddings) address all of these limitations by learning **dense, low-dimensional vector representations** where semantically similar words are close together in the vector space. In Module 4, we will explore how these representations are learned and why they revolutionized NLP.

## 9. Summary & References

### Key Takeaways

1. **Text preprocessing** (lowercasing, tokenization, stop word removal, stemming/lemmatization) is essential for cleaning raw text.
2. **Bag-of-Words** represents documents as word count vectors -- simple but effective as a baseline.
3. **TF-IDF** improves on BoW by weighting terms based on how informative they are (frequent in the document, rare in the corpus).
4. **N-grams** capture local word order (bigrams, trigrams) and can improve classification.
5. **Classical ML classifiers** (Logistic Regression) combined with TF-IDF features achieve strong baselines on text classification tasks.
6. **Limitations**: No semantic understanding, sparse representations, vocabulary mismatch. These motivate word embeddings and deep learning.

### References

- **Book**: Jurafsky, D. & Martin, J.H. *Speech and Language Processing* (3rd ed. draft). Chapters 2 (Regular Expressions, Tokenization), 4 (Naive Bayes and Sentiment), 6 (Vector Semantics). Available free online: https://web.stanford.edu/~jurafsky/slp3/
- **Book**: Bird, S., Klein, E., & Loper, E. *Natural Language Processing with Python* (NLTK Book). Available free online: https://www.nltk.org/book/
- **Course**: Stanford CS224n: Natural Language Processing with Deep Learning, Lectures 1-2. Course materials: https://web.stanford.edu/class/cs224n/
- **scikit-learn Documentation**: Text feature extraction -- https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction